In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [ ]:
llm.invoke("How will the weather be in munich today?")

In [ ]:
from langchain_core.tools import tool


@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args:
        city (str): The name of the city where you want to check the weather.

    Returns:
        str: A description of the current weather in the specified city.
    """
    return "Sunny, 22°C"

tools = [fake_weather_api]

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

In [ ]:
result.tool_calls

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? I would like to eat outside if possible"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [ ]:
tool_mapping = {
    "fake_weather_api": fake_weather_api
}
tool_mapping

In [ ]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

In [ ]:
tool_output

In [ ]:
messages

In [ ]:
llm_with_tools.invoke(messages)

LangChain also provides predefined tools

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=3)

tools = [tavily_tool]
tools[0].name

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
result = llm_with_tools.invoke("How will the weather be in munich today?")
result

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage

messages = [
    HumanMessage(
        "How will the weather be in munich today? I would like to eat outside if possible"
    )
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

In [ ]:
llm_output.tool_calls

In [ ]:
tool_mapping = {
    tools[0].name: tools[0]
}
tool_mapping

In [ ]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    # Ensure the tool output includes the 'type' field
    if isinstance(tool_output, list):
        for item in tool_output:
            item['type'] = 'text'
    messages.append(ToolMessage(content=tool_output, tool_call_id=tool_call["id"]))

In [ ]:
tool_output

In [ ]:
messages

In [ ]:
llm_with_tools.invoke(messages)